<a href="https://colab.research.google.com/github/sonicchomp/MyImplentetion/blob/master/Convex%20Fomulations%20of%20Learning%20from%20CrowdSourcing/CFoLF.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

https://www.jstage.jst.go.jp/article/tjsai/27/3/27_3_133/_pdf/-char/ja


In [0]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [0]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
import theano
import theano.tensor as T


In [0]:
class Optimizer(object):
  def __init__(self,params=None):
    if params is None:
      return NotImplementedError()
    self.params = params
  def updates(self,loss=None):
    if loss is None:
      return NotImplementedError()
    self.updates = OrderedDict()
    self.gparams = [T.grad(loss,param) for param in self.params]


In [0]:
class SGD(Optimizer):
    def __init__(self, learning_rate=0.01, params=None):
        super(SGD, self).__init__(params=params)
        self.learning_rate = 0.01

    def updates(self, loss=None):
        super(SGD, self).updates(loss=loss)

        for param, gparam in zip(self.params, self.gparams):
            self.updates[param] = param - self.learning_rate * gparam

        return self.updates

In [0]:
def make_W(w_0, lambd, j):
    W = np.empty((j,w_0.shape[0]))

    for i in range(j):
        w_j = np.random.normal(loc=w_0,scale=1.0,size=w_0.shape[0])
        W[i] = w_j
    return W

In [0]:
def model(X,y,lambd, w_0,W):
  X = T.matrix(name="X")
  y = T.vector(name="y")  
  w_0 = theano.shared(w_0, name="w_0")
  W = theano.shared(W, name="W")


  p_1 = 1/(1+T.exp(-T.dot(X,w)))
  xent = -y * T.log(p_1) - (1-y)*T.log(1-p_1)
  loss = xent.mean() + lambd * (w ** 2).sum()/2

  params = [w]
  updates = SGD(params=params).updates(loss)

  print('start: compile model')

  train = theano.function(
            inputs=[X, y],
            outputs=[loss,w],
            updates=updates)

  print('complete: compile model')

  return train

In [0]:
def main():
    read_file = '/content/drive/My Drive/論文実装/Convex Fomulations of Learning from CrowdSourcing/winequality-red.csv'
    data = pd.read_csv(read_file, sep=';')
    X = data.drop(data.columns[-1],axis=1)
    y = data[data.columns[-1]]

    X_train, X_test, y_train, y_test = train_test_split(X,y,shuffle = True)

    eta = 0.01
    lambd = 0.01
    training_epochs = 10
    np.random.seed(seed=38)
    w_0 = np.random.normal(loc=0.0,scale=eta,size=X_train.shape[1])
    W = make_W(w_0,lambd,X_train.shape[0])

    pass

Pr[w_0|\eta] = N(0,\eta^-1 I]  
Pr[w_j|w_o,lambd] =  N(w_0,\lambd I]  
Pr[y|x_i, w_j]

In [0]:
main()

(1199, 11)